# BLAST

This example illustrates how to perform meta-blocking by using BLAST.

BLAST can be only used on clean-clean datasets (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [3]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 7.0 MB/s eta 0:00:012
     |████████████████████████████████| 198 kB 12.5 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3569006b21628eca4e103e7c64562fe10ce92d74bd35e7a3af49628bce082d76
  Stored in directory: /home/aditi/.cache/pip/wheels/52/45/50/69db7b6e1da74a1b9fcc097827db9185cb8627117de852731e
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyspark
import sparker
import random

## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.
*source_id* is used to identify from which dataset the profile belongs, it is necessary for the attributes alignment

In [2]:
# Profiles contained in the first dataset
profiles1 = sparker.CSVWrapper.load_profiles('/Final_Project/sample/Sample1_cleaned.csv', 
                                              real_id_field = "id", 
                                              source_id=1)
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

2022-04-09 19:32:19,975 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-04-09 19:32:19,976 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-09 19:32:20,352 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
separator_ids

[29]

Let's visualize a profile to check if they are correctly loaded

In [4]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'DisplayName', 'value': 'Community'}, {'key': 'Location', 'value': 'on the server farm'}, {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."}, {'key': 'Age', 'value': '27'}, {'key': 'phone', 'value': '655-452-4816'}], 'original_id': '0', 'source_id': 1}


Loads the second dataset and extract the max id (it will be used later)

In [5]:
profiles2 = sparker.CSVWrapper.load_profiles('/Final_Project/sample/Sample1_cleaned.csv', 
                                              start_id_from = separator_id+1, 
                                              real_id_field = "id", 
                                              source_id=2)
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

Finally, concatenate the two RDDs

In [6]:
profiles = profiles1.union(profiles2)

In [7]:
profiles.toDF().show(4590) #b000jz4hqo

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        1|
|[{DisplayName, Ge...|          1|         1|        1|
|[{DisplayName, Ji...|          2|         2|        1|
|[{DisplayName, St...|          3|         3|        1|
|[{DisplayName, tx...|          4|         4|        1|
|[{DisplayName, Ke...|          5|         5|        1|
|[{DisplayName, Em...|          6|         6|        1|
|[{DisplayName, To...|          7|         7|        1|
|[{DisplayName, Se...|          8|         8|        1|
|[{DisplayName, Fr...|          9|         9|        1|
|[{DisplayName, Br...|         10|        10|        1|
|[{DisplayName, cm...|         11|        11|        1|
|[{DisplayName, kā...|         12|        12|        1|
|[{DisplayName, Sc...|         13|        13|        1|
|[{DisplayName, Ma...|         14|        14|   

### Groundtruth (optional)
If you have a groundtruth you can measure the performance at each blocking step.

When you load the groundtruth, it contains the original profiles IDs, so it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [8]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/sample/ground_truth_sample.csv', 'id1', 'id2')

In [9]:
gt.toDF().show(1400)

+---------------+----------------+
|first_entity_id|second_entity_id|
+---------------+----------------+
|             20|               0|
|             21|               1|
|             22|               2|
|             23|               3|
|             24|               4|
|             25|               5|
|             26|               6|
|             27|               7|
|             28|               8|
|             29|               9|
+---------------+----------------+



In [10]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [11]:
new_gt

{(20, 30),
 (21, 31),
 (22, 32),
 (23, 33),
 (24, 34),
 (25, 35),
 (26, 36),
 (27, 37),
 (28, 38),
 (29, 39)}

In [12]:
# We can explore some pairs
random.sample(new_gt, 2)

/tmp/ipykernel_6899/3362377442.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(new_gt, 2)


[(26, 36), (25, 35)]

## Attributes alignment

BLAST employs LSH to automatically align the attributes.

If *compute_entropy* is set to True it computes the entropy of each cluster of attributes that can be used to further improve the meta-blocking performance.

*target_threshold* parameter regulates the similarity that the values of the attributes should have to be clustered together

In [13]:
clusters = sparker.AttributeClustering.cluster_similar_attributes(profiles,
                                  num_hashes=128,
                                  target_threshold=0.5,
                                  compute_entropy=True)
clusters

[{'cluster_id': 0, 'keys': ['2_AboutMe', '1_AboutMe'], 'entropy': 8.231411761241931},
 {'cluster_id': 1, 'keys': ['2_Country', '1_Country'], 'entropy': 1.0},
 {'cluster_id': 2, 'keys': ['2_City', '1_City'], 'entropy': 4.875145808605837},
 {'cluster_id': 3, 'keys': ['1_Location', '2_Location'], 'entropy': 5.24382798408352},
 {'cluster_id': 4, 'keys': ['2_Age', '1_Age'], 'entropy': 3.8980685120588388},
 {'cluster_id': 5, 'keys': ['2_phone', '1_phone'], 'entropy': 6.416798790750068},
 {'cluster_id': 6, 'keys': ['2_State', '1_State'], 'entropy': 3.8518232255517666},
 {'cluster_id': 7, 'keys': ['1_DisplayName', '2_DisplayName'], 'entropy': 5.285402218862247},
 {'cluster_id': 8, 'keys': [], 'entropy': 1.0}]

## Blocking
Now we can perform blocking by using the generated clusters

In [14]:
blocks = sparker.Blocking.create_blocks_clusters(profiles, clusters, separator_ids)
print("Number of blocks",blocks.count())

Number of blocks 777


In [15]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0}, {30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0}, {30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0}, {30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0}, {30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0, 3}, {33, 30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 3, 8, 9, 10, 12, 14, 15, 17, 24, 25, 26, 28, 29}, {33, 38, 39, 40, 42, 44, 45, 47, 54, 55, 56, 58, 59, 30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0}, {30}], 'entropy': 5.24382798408352, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 7, 'profiles': [{0}, {30}], 'entropy': 8.231411761241931, 'cluster_id': -1, 'blocking_key': ''},
 {'bl

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [16]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.005)

In [17]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [18]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 0.1
Precision 0.010869565217391304
Number of comparisons 92


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.


For every partition of the RDD, the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [19]:
sc = SparkContext.getOrCreate()

NameError: name 'SparkContext' is not defined

In [23]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

block_entropies = sc.broadcast(blocks.map(lambda b : (b.block_id, b.entropy)).collectAsMap())

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Meta-blocking with BLAST
BLAST employs $\chi^2$ weighting scheme.

*chi2divider* parameter regulates the pruning aggressivity, a lower value performs a more aggressive pruning.

In [24]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CHI_SQUARE,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          use_entropy=True,
                          blocks_entropies=block_entropies,
                          chi2divider=2.0
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9986510791366906
Precision 0.31924680178237747
Number of comparisons 6957


## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [25]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(0, 2733, 474746.74542385637),
 (404, 4257, 323134.03437346633),
 (404, 4292, 377061.62799117394),
 (1448, 2689, 28001.924248551786),
 (1448, 3092, 533286.1049898568),
 (2612, 2818, 9460.396018690539),
 (2412, 3616, 634811.297962843),
 (2412, 3037, 24872.747932579347),
 (1284, 2686, 859823.5641014528),
 (1284, 3052, 65034.464088759465)]